In [30]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series

dtype = {
    'id': np.int8,
    'qid1': np.int8,
    'qid2': np.int8,
    'question1': np.str,
    'question2': np.str,
    'is_duplicate': np.int8
}
train = pd.read_csv('/input/Kaggle/Quora/train.csv', dtype=dtype)
print(train.shape)
train.dropna(inplace=True)
print(train.shape)

(404290, 6)
(404288, 6)


In [2]:
dtype = {
    'test_id': np.int8,
    'question1': np.str,
    'question2': np.str
}
test = pd.read_csv('/input/Kaggle/Quora/test.csv', dtype=dtype)
print( (test.isnull()).sum().sum() )
test.replace(np.nan, '', inplace=True)
print( (test.isnull()).sum().sum() )

6
0


In [297]:
train.head(1)

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0


# 数据清洗

In [31]:
# 必要的
def lower(q):
    return q.lower()

train['question1_refine'] = train['question1'].apply(lower)
train['question2_refine'] = train['question2'].apply(lower)

train.head(1)

,id,qid1,qid2,question1,question2,is_duplicate,question1_refine,question2_refine
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...


In [14]:
# 结果验证这个不好，但从逻辑上，不好理解。可能是由于这样把很多的表单符号单独出来了，不过后面排除掉，还是效果欠佳
import re

pattern1 = re.compile(r'(?<=\w)([\?\.,\)])')
pattern2 = re.compile(r'([\(])(?=\w)')

def fill_blank(matched):
    return ' {} '.format(matched.group(1))

def leave_blank(q):
    q = re.sub(pattern1, fill_blank, q)
    q = re.sub(pattern2, fill_blank, q)
    return q

train['question1_refine'] = train['question1_refine'].apply(leave_blank)
train['question2_refine'] = train['question2_refine'].apply(leave_blank)

train.head(1)

,id,qid1,qid2,question1,question2,is_duplicate,question1_refine,question2_refine,q1_len,q2_len,q1_set_len,q2_set_len,common_len,common_ratio
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,what is the step by step guide to invest in sh...,what is the step by step guide to invest in sh...,14.0,12.0,12.0,11.0,10.0,0.833333


In [25]:
# 经过测试，这个效果也不好
import re

remove_mark_pattern = re.compile(r'\W')
def remove_mark(q):
    return re.sub(remove_mark_pattern, ' ', q)

train['question1_refine'] = train['question1_refine'].apply(remove_mark)
train['question2_refine'] = train['question2_refine'].apply(remove_mark)

train.head(1)

,id,qid1,qid2,question1,question2,is_duplicate,question1_refine,question2_refine,q1_len,q2_len,q1_set_len,q2_set_len,common_len,common_ratio
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,what is the step by step guide to invest in sh...,what is the step by step guide to invest in sh...,14.0,12.0,12.0,11.0,10.0,0.833333


In [17]:
# 尝试过，结果还是不尽如人意
def replace(q, replacements=()):
    for f,t in replacements:
        q = q.replace(f, t)
    return q

replacements = (("'s", ''), ('"', ''), ("/", ' '), ("'m", ' am'), ("'ve", ' have'), ("'ll", ' will'))
train['question1_refine'] = train['question1_refine'].apply(replace, replacements=replacements)
train['question2_refine'] = train['question2_refine'].apply(replace, replacements=replacements)

train.head(1)

,id,qid1,qid2,question1,question2,is_duplicate,question1_refine,question2_refine,q1_len,q2_len,...,q1_start_with_can,q2_start_with_can,q1_start_with_should,q2_start_with_should,q1_start_with_will,q2_start_with_will,q1_start_with_is,q2_start_with_is,q1_start_with_do,q2_start_with_do
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,what is the step by step guide to invest in sh...,what is the step by step guide to invest in sh...,14.0,12.0,...,0,0,0,0,0,0,0,0,0,0


# Feature Engineering

```
STOPWORDS = set(('i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', 'couldn', 'didn', 'doesn', 'hadn', 'hasn', 'haven', 'isn', 'ma', 'mightn', 'mustn', 'needn', 'shan', 'shouldn', 'wasn', 'weren', 'won', 'wouldn'))
VOCAB_SET = set(word2vec.vocab.keys())
```

In [32]:
# 结果验证有效果
if False:
    train.drop(['q1_len', 'q2_len', 'q1_set_len', 'q2_set_len', 'common_len', 'common_ratio'],
              axis=1, inplace=True)
SMOOTH = 1

def word_count(row):
    q1_list = row['question1_refine'].split()
    q2_list = row['question2_refine'].split()
    
    q1_len = len(q1_list)
    q2_len = len(q2_list)
    
    # set
    q1_set = set(q1_list)
    q2_set = set(q2_list)
    common_set = q1_set.intersection(q2_set)
    
    q1_set_len = len(q1_set)
    q2_set_len = len(q2_set)
    common_len = len(common_set)
    common_ratio = (2 * common_len) / (q1_set_len + q2_set_len + SMOOTH)
    
    return Series(data=(q1_len, q2_len, q1_set_len, q2_set_len, common_len, common_ratio),
              index=['q1_len', 'q2_len', 'q1_set_len', 'q2_set_len', 'common_len', 'common_ratio'])

    # 后面的部分尝试效果欠佳，虽然逻辑上好像符合
#     q1_entity_set = (q1_set - STOPWORDS)#.intersection(VOCAB_SET)
#     q2_entity_set = (q2_set - STOPWORDS)#.intersection(VOCAB_SET)
#     q1_entity_set = q1_set.intersection(VOCAB_SET)
#     q2_entity_set = q2_set.intersection(VOCAB_SET)
#     entity_common_set = q1_entity_set.intersection(q2_entity_set)
    
#     # ns: non-stop
#     q1_entity_len = len(q1_entity_set)
#     q2_entity_len = len(q2_entity_set)
#     entity_common_len = len(entity_common_set)
#     entity_common_ratio = (2 * entity_common_len) / (q1_entity_len + q2_entity_len + SMOOTH)

#     return Series(data=(q1_len, q2_len, q1_set_len, q2_set_len, common_len, common_ratio,
#                         q1_entity_len, q2_entity_len, entity_common_len, entity_common_ratio),
#                   index=['q1_len', 'q2_len', 'q1_set_len', 'q2_set_len', 'common_len', 'common_ratio',
#                         'q1_entity_len', 'q2_entity_len', 'entity_common_len', 'entity_common_ratio'])

word_counts = train.apply(word_count, axis=1)
train = pd.concat([train, word_counts], axis=1)

train.head(1)

,id,qid1,qid2,question1,question2,is_duplicate,question1_refine,question2_refine,q1_len,q2_len,q1_set_len,q2_set_len,common_len,common_ratio
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,14.0,12.0,12.0,11.0,10.0,0.833333


In [7]:
# 经过验证，这个是有效果的
from nltk.util import ngrams
SMOOTH = 1

def ngrams_count(row, n=2, suffix=''):
    q1_ngrams = ngrams(row['question1_refine'].split(), n)
    q2_ngrams = ngrams(row['question2_refine'].split(), n)
    
    q1_ngram_set = set([' '.join(tup) for tup in q1_ngrams])
    q2_ngram_set = set([' '.join(tup) for tup in q2_ngrams])
    common_set = q1_ngram_set.intersection(q2_ngram_set)
    
    q1_len = len(q1_ngram_set)
    q2_len = len(q2_ngram_set)
    common_len = len(common_set)
    common_ratio = (2 * common_len) / (q1_len + q2_len + SMOOTH)
    
    return Series(data=(common_len, common_ratio),
              index=['common_len'+suffix, 'common_ratio'+suffix])

In [8]:
grams_2 = train.apply(ngrams_count, axis=1, n=2, suffix='_2_grams')
train = pd.concat([train, grams_2], axis=1)

train.head(1)

,id,qid1,qid2,question1,question2,is_duplicate,question1_refine,question2_refine,q1_len,q2_len,q1_set_len,q2_set_len,common_len,common_ratio,common_len_2_grams,common_ratio_2_grams
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,what is the step by step guide to invest in sh...,what is the step by step guide to invest in sh...,14.0,12.0,12.0,11.0,10.0,0.833333,10.0,0.8


In [10]:
grams_3 = train.apply(ngrams_count, axis=1, n=3, suffix='_3_grams')
train = pd.concat([train, grams_3], axis=1)

train.head(1)

,id,qid1,qid2,question1,question2,is_duplicate,question1_refine,question2_refine,q1_len,q2_len,q1_set_len,q2_set_len,common_len,common_ratio,common_len_2_grams,common_ratio_2_grams,common_len_3_grams,common_ratio_3_grams
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,what is the step by step guide to invest in sh...,what is the step by step guide to invest in sh...,14.0,12.0,12.0,11.0,10.0,0.833333,10.0,0.8,9.0,0.782609


In [12]:
# 结果验证有效果
def start_with(q, symbols):
    s = q.split()
    return int(s[0] in symbols) if len(s) > 0 else 0

train['q1_start_with_what'] = train['question1_refine'].apply(start_with, symbols=('what', "what's"))
train['q2_start_with_what'] = train['question2_refine'].apply(start_with, symbols=('what', "what's"))

train['q1_start_with_how'] = train['question1_refine'].apply(start_with, symbols=('how'))
train['q2_start_with_how'] = train['question2_refine'].apply(start_with, symbols=('how'))

train['q1_start_with_why'] = train['question1_refine'].apply(start_with, symbols=('why'))
train['q2_start_with_why'] = train['question2_refine'].apply(start_with, symbols=('why'))

train['q1_start_with_who'] = train['question1_refine'].apply(start_with, symbols=('who'))
train['q2_start_with_who'] = train['question2_refine'].apply(start_with, symbols=('who'))

train['q1_start_with_where'] = train['question1_refine'].apply(start_with, symbols=('where'))
train['q2_start_with_where'] = train['question2_refine'].apply(start_with, symbols=('where'))

train['q1_start_with_when'] = train['question1_refine'].apply(start_with, symbols=('when'))
train['q2_start_with_when'] = train['question2_refine'].apply(start_with, symbols=('when'))

train['q1_start_with_which'] = train['question1_refine'].apply(start_with, symbols=('which'))
train['q2_start_with_which'] = train['question2_refine'].apply(start_with, symbols=('which'))

train['q1_start_with_can'] = train['question1_refine'].apply(start_with, symbols=('can', 'could'))
train['q2_start_with_can'] = train['question2_refine'].apply(start_with, symbols=('can', 'could'))

train['q1_start_with_should'] = train['question1_refine'].apply(start_with, symbols=('should'))
train['q2_start_with_should'] = train['question2_refine'].apply(start_with, symbols=('should'))

train['q1_start_with_will'] = train['question1_refine'].apply(start_with, symbols=('will', 'would'))
train['q2_start_with_will'] = train['question2_refine'].apply(start_with, symbols=('will', 'would'))

train['q1_start_with_is'] = train['question1_refine'].apply(start_with, symbols=('is', 'am', 'are'))
train['q2_start_with_is'] = train['question2_refine'].apply(start_with, symbols=('is', 'am', 'are'))

train['q1_start_with_do'] = train['question1_refine'].apply(start_with, symbols=('do', 'does', 'did'))
train['q2_start_with_do'] = train['question2_refine'].apply(start_with, symbols=('do', 'does', 'did'))

train.head(1)

,id,qid1,qid2,question1,question2,is_duplicate,question1_refine,question2_refine,q1_len,q2_len,...,q1_start_with_can,q2_start_with_can,q1_start_with_should,q2_start_with_should,q1_start_with_will,q2_start_with_will,q1_start_with_is,q2_start_with_is,q1_start_with_do,q2_start_with_do
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,what is the step by step guide to invest in sh...,what is the step by step guide to invest in sh...,14.0,12.0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
# 最终验证没有效果
# train.drop(['q1_has_question_mark', 'q2_has_question_mark'], axis=1, inplace=True)

def has_symbol(q, symbol):
    return 1 if symbol in q else 0

train['q1_has_question_mark'] = train['question1_refine'].apply(has_symbol, symbol='?')
train['q2_has_question_mark'] = train['question2_refine'].apply(has_symbol, symbol='?')

train.head(1)

,id,qid1,qid2,question1,question2,is_duplicate,question1_refine,question2_refine,q1_len,q2_len,...,q1_start_with_should,q2_start_with_should,q1_start_with_will,q2_start_with_will,q1_start_with_is,q2_start_with_is,q1_start_with_do,q2_start_with_do,q1_has_question_mark,q2_has_question_mark
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,what is the step by step guide to invest in sh...,what is the step by step guide to invest in sh...,14.0,12.0,...,0,0,0,0,0,0,0,0,1,1


## 辅助Error Analysis

In [35]:
from gensim.models.word2vec import Word2Vec, KeyedVectors
word2vec = KeyedVectors.load_word2vec_format('/input/Kaggle/Word2Vec/GoogleNews-vectors-negative300.bin.gz', binary=True)

Using TensorFlow backend.


In [ ]:
unique_words = {}

for q in train['question1_refine']:
    s = q.split()
    for word in s:
        if word in unique_words:
            unique_words[word] += 1
        else:
            unique_words[word] = 1

for q in train['question2_refine']:
    s = q.split()
    for word in s:
        if word in unique_words:
            unique_words[word] += 1
        else:
            unique_words[word] = 1
            
unique_words_list = [ (v,k) for k,v in unique_words.items() ]
unique_words_list = sorted(unique_words_list, reverse=True)

rare_word_list = []

for freq, word in unique_words_list:
    if word not in word2vec.vocab:
        rare_word_list.append((freq, word))

In [396]:
unique_words_list[:500]

[(841173, '?'),
 (376550, 'the'),
 (311497, 'what'),
 (269535, 'is'),
 (220398, 'how'),
 (214430, 'i'),
 (211154, 'a'),
 (205263, 'to'),
 (196454, 'in'),
 (160811, 'do'),
 (159720, 'of'),
 (145692, 'are'),
 (133192, 'and'),
 (114003, 'can'),
 (104343, 'for'),
 (100283, ','),
 (88922, 'you'),
 (83681, 'why'),
 (71511, '.'),
 (70838, 'my'),
 (70363, 'best'),
 (68830, 'it'),
 (60447, 'on'),
 (56318, 'does'),
 (44005, 'or'),
 (43931, 'which'),
 (43722, 'be'),
 (43674, 'if'),
 (41670, 'some'),
 (40753, 'have'),
 (40365, 'that'),
 (39673, 'with'),
 (39476, 'get'),
 (38976, 'should'),
 (36899, 'an'),
 (35214, 'from'),
 (33872, 'your'),
 (28320, 'india'),
 (28173, 'will'),
 (26149, 'when'),
 (25969, 'people'),
 (25564, 'who'),
 (25444, 'like'),
 (24983, '('),
 (24964, 'at'),
 (24893, ')'),
 (24659, 'good'),
 (23531, 'would'),
 (23337, 'there'),
 (22593, 'as'),
 (21131, 'about'),
 (20332, 'not'),
 (20266, 'between'),
 (19177, 'one'),
 (18890, 'most'),
 (18254, 'we'),
 (18108, 'make'),
 (18035, 

In [397]:
rare_word_list[:500]

[(841173, '?'),
 (211154, 'a'),
 (205263, 'to'),
 (159720, 'of'),
 (133192, 'and'),
 (100283, ','),
 (71511, '.'),
 (24983, '('),
 (24893, ')'),
 (17854, 'quora'),
 (7554, '2016'),
 (6657, '500'),
 (6428, '1000'),
 (6193, "i'm"),
 (5077, 'instagram'),
 (3829, '10'),
 (3272, '2017'),
 (2840, 'whatsapp'),
 (1818, '2000'),
 (1613, '000'),
 (1597, 'snapchat'),
 (1590, '-'),
 (1419, '2015'),
 (1351, "someone's"),
 (1333, ':'),
 (1300, 'jio'),
 (1250, '"'),
 (1236, '20'),
 (1161, 'narendra'),
 (1156, '12'),
 (1103, 'nra'),
 (1076, '30'),
 (1057, '15'),
 (1037, '/'),
 (1025, "i've"),
 (995, '100'),
 (943, 'favourite'),
 (858, '16'),
 (857, 'upsc'),
 (856, 'pokémon'),
 (840, 'sbi'),
 (839, "trump's"),
 (778, '12th'),
 (773, '50'),
 (773, '"the'),
 (738, '18'),
 (726, 'spotify'),
 (709, '2014'),
 (696, 'work-life'),
 (669, '13'),
 (667, 'mind-blowing'),
 (658, 'tcs'),
 (655, 'and/or'),
 (644, '14'),
 (637, "india's"),
 (635, '25'),
 (625, 'mbbs'),
 (616, '11'),
 (612, 'cgl'),
 (601, 'c++?'),
 (

# 训练模型

In [33]:
train.columns

Index(['id', 'qid1', 'qid2', 'question1', 'question2', 'is_duplicate',
       'question1_refine', 'question2_refine', 'q1_len', 'q2_len',
       'q1_set_len', 'q2_set_len', 'common_len', 'common_ratio'],
      dtype='object')

In [34]:
train.head(1)

,id,qid1,qid2,question1,question2,is_duplicate,question1_refine,question2_refine,q1_len,q2_len,q1_set_len,q2_set_len,common_len,common_ratio
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,14.0,12.0,12.0,11.0,10.0,0.833333


In [ ]:
import xgboost as xgb
# from sklearn.preprocessing import StandardScaler

X_all = train.drop(['id', 'qid1', 'qid2', 'question1', 'question2', 'is_duplicate' ,'question1_refine', 'question2_refine'], axis=1)
# X_all = StandardScaler().fit_transform(X_all)# doesn't work
y_all = train.is_duplicate

params = {
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    # 自动设置的参数比简单自己调整的要效果好，这里没有做parameter tunning，因为这次实验的重点不在这
#     'eta': 0.02, # learning rate
#     'gamma': 0.01, # minimum loss reduction required to make a further partition on a leaf node of the tree
#     'max_depth': 16, #maximum depth of a tree
#     'min_child_weight': 1, #minimum sum of instance weight (hessian) needed in a child.
#     'silent': 0, #0 means printing running messages, 1 means silent mode.
}

dtrain = xgb.DMatrix(X_all, label=y_all)

# 等待early stopping
history = xgb.cv(params, dtrain, num_boost_round=1024, early_stopping_rounds=5, verbose_eval=20)
print(history.loc[history.shape[0]-1, ['test-logloss-mean', 'train-logloss-mean']])

# Trial 1
# init 0.492954	
# 符号分割: 0.508782 bad
# 去掉符号: 0.514202 even bad
# 2 grams: 0.502062 good
# 3 grams: 0.499647 good

# Trial 2
# init 0.492954
# 2 gram, 0.482699 good
# 3 gram,0.480798 good
# Start with: 0.444659 quite good
# question_mark，bad

# Trial 3
# init, replace, 0.494482, bad, compared with Trial 1/2

# Trial 4 
# init, remove_mark: 0.514202, quite bad

In [ ]:
booster = xgb.train(params, dtrain)
xgb.plot_importance(booster=booster)

In [ ]:
xgb.plot_tree(booster=booster)

In [ ]:
xgb.to_graphviz(booster=booster)